In [1]:
from rfgap import RFGAP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from dataset import dataprep, load_regression
from phate import PHATE

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import random
import json
import multiprocessing
from joblib import Parallel, delayed
import os

from quantile_forest import RandomForestQuantileRegressor

In [2]:
data_name = 'AutoMPG'
x, y = load_regression('../datasets/regression/' + data_name)
n, d   = x.shape

level = 0.95

In [3]:
x_train, x_test, y_train, y_test, inds_train, inds_test = train_test_split(x, y, np.arange(n),
                                                                               test_size = .2, 
                                                                               random_state = 42)

In [4]:
upr_quantile = 1 - (1 - level) / 2
lwr_quantile = (1 - level) / 2

qrf = RandomForestQuantileRegressor(random_state = 42)
qrf.fit(x_train, y_train)

lwr = qrf.predict(x_test, quantiles = lwr_quantile)
med = qrf.predict(x_test, quantiles = 0.5)
upr = qrf.predict(x_test, quantiles = upr_quantile)

print('qrf types: ', type(lwr))


oob_score = qrf.predict(x_train, quantiles = 0.5, oob_score = True)

qrf types:  <class 'numpy.ndarray'>


In [7]:
oob_score

array([-0.54399927,  0.13505197, -0.31337809, -1.21023822,  1.73658791,
       -0.95399247,  1.87752307,  0.45535916, -0.36462724, -0.95399247,
       -0.95399247,  0.37848543,  0.40411001,  1.0319121 ,  0.71160491,
       -0.56962384, -0.44150097, -0.28134737, -1.46648397,  0.07099053,
       -0.31337809, -1.3383611 , -0.69774672, -1.21023822,  0.45535916,
        1.41628072,  1.10878582,  1.99283366, -0.05713234, -0.50556241,
       -0.82586959,  1.86471078,  1.57002817, -0.05713234, -0.18525522,
        1.82627392,  0.58348203, -0.71055901, -0.95399247, -0.05713234,
       -0.31337809,  0.76285406, -0.31337809, -0.82586959,  1.73658791,
        0.13505197,  0.26317484,  1.09597353,  0.71160491, -0.76180816,
        1.41628072,  0.96785066,  0.26317484, -0.18525522,  1.31378242,
       -1.01805391,  0.32723628,  0.07099053,  1.76221248, -0.69774672,
       -0.63368528, -1.08211534, -0.44150097,  0.9294138 , -0.31337809,
       -0.18525522,  0.45535916,  0.19911341, -1.21023822, -0.36